In [ ]:
!pip install transformers

In [ ]:
# Setting up the device for GPU usage
import torch
from torch import cuda
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel



device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

from google.colab import drive
drive.mount('/content/drive')

cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_data = pd.read_csv('/content/drive/MyDrive/datasets/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/datasets/test.csv')
#train_data_y = pd.read_csv('test_labels.csv')
#Removing id column and preparing labels into the single list column
train_data.drop(['id'], inplace=True, axis=1)
train_data['labels'] = train_data.iloc[:, 1:].values.tolist()
train_data.drop(train_data.columns.values[1:-1].tolist(), inplace=True, axis=1)
train_data

,comment_text,labels
0,Explanation\nWhy the edits made under my usern...,"[0, 0, 0, 0, 0, 0]"
1,D'aww! He matches this background colour I'm s...,"[0, 0, 0, 0, 0, 0]"
2,"Hey man, I'm really not trying to edit war. It...","[0, 0, 0, 0, 0, 0]"
3,"""\nMore\nI can't make any real suggestions on ...","[0, 0, 0, 0, 0, 0]"
4,"You, sir, are my hero. Any chance you remember...","[0, 0, 0, 0, 0, 0]"
...,...,...
159566,""":::::And for the second time of asking, when ...","[0, 0, 0, 0, 0, 0]"
159567,You should be ashamed of yourself \n\nThat is ...,"[0, 0, 0, 0, 0, 0]"
159568,"Spitzer \n\nUmm, theres no actual article for ...","[0, 0, 0, 0, 0, 0]"
159569,And it looks like it was actually you who put ...,"[0, 0, 0, 0, 0, 0]"


In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
#Data Cleaning
train_data["comment_text"] = train_data["comment_text"].str.lower()
train_data["comment_text"] = train_data["comment_text"].str.replace("\xa0", " ", regex=False).str.split().str.join(" ")

In [ ]:
# training Parameters
# MAX_LEN = 512
# TRAIN_BATCH_SIZE = 16
# EPOCHS = 1
# LEARNING_RATE = 1e-05
# NUM_WORKERS = 2
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [ ]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [ ]:
#train_data_sampled = train_data.sample(n=5000)
#train_data_sampled.to_csv('/content/drive/MyDrive/datasets/5000_train.csv')

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len: int, eval_mode: bool = False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.text = dataframe.comment_text
        self.eval_mode = eval_mode
        if self.eval_mode is False:
            self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text.iloc[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        output = {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

        if self.eval_mode is False:
            output['targets'] = torch.tensor(self.targets.iloc[index], dtype=torch.float)

        return output

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
xtrain=train_data.sample(frac=train_size,random_state=200)
xtest=train_data.drop(xtrain.index).reset_index(drop=True)
xtrain = xtrain.reset_index(drop=True)
xtrain.to_csv('/content/drive/MyDrive/datasets/5000_train.csv')
xtest.to_csv('/content/drive/MyDrive/datasets/1000_test.csv')


print("FULL Dataset: {}".format(train_data.shape))
print("TRAIN Dataset: {}".format(xtrain.shape))
print("TEST Dataset: {}".format(xtest.shape))

training_set = MultiLabelDataset(xtrain, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(xtest, tokenizer, MAX_LEN)

FULL Dataset: (159571, 2)
TRAIN Dataset: (127657, 2)
TEST Dataset: (31914, 2)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
#training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)


In [ ]:
#cleaning dataloader
# train_params = {'batch_size': TRAIN_BATCH_SIZE,
#                 'shuffle': True,
#                 'num_workers': NUM_WORKERS
#                 }
# training_loader = DataLoader(training_set, **train_params)

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model.

class DistilBERTClass(torch.nn.Module):

    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 6)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
#Loading Neural Network model
model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [ ]:
#Loss function and optimizer
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
#Fine tune the model
def train(model, optimizer, epoch):

    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(model, optimizer, epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific v

Epoch: 0, Loss:  0.7071337699890137


5it [00:08,  1.19s/it]

Epoch: 0, Loss:  0.6277883648872375


10it [00:12,  1.18it/s]

...

7970it [1:41:17,  1.32it/s]

Epoch: 0, Loss:  0.01315651647746563


7975it [1:41:20,  1.32it/s]

Epoch: 0, Loss:  0.043401189148426056


7979it [1:41:23,  1.31it/s]


In [ ]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.5

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific v

In [ ]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.9509410707944268
Hamming Loss = 0.015160535606108081


In [ ]:
# test_data = pd.read_csv('/content/drive/MyDrive/datasets/test.csv')
# test_data.head()

In [ ]:
# test_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN, eval_mode = True)
# testing_params = {'batch_size': TRAIN_BATCH_SIZE,
#                'shuffle': False,
#                'num_workers': 2
#                 }
# test_loader = DataLoader(test_set, **testing_params)

In [ ]:
# all_test_pred = []

# def test(epoch):
#     model.eval()

#     with torch.inference_mode():

#         for _, data in tqdm(enumerate(test_loader, 0)):


#             ids = data['ids'].to(device, dtype=torch.long)
#             mask = data['mask'].to(device, dtype=torch.long)
#             token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
#             outputs = model(ids, mask, token_type_ids)
#             probas = torch.sigmoid(outputs)

#             all_test_pred.append(probas)
#     return probas

In [ ]:
#probas = test(model)

In [ ]:
#all_test_pred = torch.cat(all_test_pred)

In [ ]:
# def evaluate(model, loss_function, test_dataloader, device):
#     model.eval()
#     test_loss, test_r2 = [], []
#     for batch in test_dataloader:
#         batch_inputs, batch_masks, batch_labels = \
#                                  tuple(b.to(device) for b in batch)
#         with torch.no_grad():
#             outputs = model(batch_inputs, batch_masks)
#         loss = loss_function(outputs, batch_labels)
#         test_loss.append(loss.item())
#         r2 = r2_score(outputs, batch_labels)
#         test_r2.append(r2.item())
#     return test_loss, test_r2


In [ ]:
# def r2_score(outputs, labels):
#     labels_mean = torch.mean(labels)
#     ss_tot = torch.sum((labels - labels_mean) ** 2)
#     ss_res = torch.sum((labels - outputs) ** 2)
#     r2 = 1 - ss_res / ss_tot
#     return r2

In [ ]:
# submit_df = test_data.copy()
# submit_df.drop("comment_text", inplace=True, axis=1)

In [ ]:
#label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [ ]:
# for i,name in enumerate(label_columns):

#     submit_df[name] = all_test_pred[:, i].cpu()
#     submit_df.head()

In [ ]:
# submit_df.to_csv('submission.csv', index=False)
# submit_df.head()